<a href="https://colab.research.google.com/github/jjgranadat/Semillero_SOFA/blob/Isabella/SVM_KNN_16QAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Generación y modulación

In [ ]:
Ns= 1000
Mod=16 #16QAM
Symbols_tx = np.random.randint(Mod, size=Ns)
df_sym_tx= pd.DataFrame(Symbols_tx)
print(Symbols_tx[0:10])

In [ ]:
mod_dict = {0:-3+3j, #0000
            1:-3+1j, #0001
            2:-3-3j, #0010
            3:-3-1j, #0011
            4:-1+3j, #0100
            5:-1+1j, #0101
            6:-1-3j, #0110
            7:-1-1j, #0111
            8:3+3j, #1000
            9:3+1j, #1001
            10:3-3j, #1010
            11:3-1j, #1011
            12:1+3j, #1100
            13:1+1j, #1101
            14:1-3j, #1110
            15:1-1j} #1111

In [ ]:
#Modulación reemplazando símbolos por valores complejos - mapeo (opción 2)
df_sym_mod2 = df_sym_tx.iloc[:,0].map(mod_dict)
df_sym_mod2.head(20)

# Transmisión canal AWGN

In [ ]:
Xr = np.real(df_sym_mod2)
Xi = np.imag(df_sym_mod2)

In [ ]:
# Adición de ruido blanco Gaussiano (AWGN)

target_snr_db = 16
Xr_avg_p = np.mean(Xr**2)
Xr_avg_db = 10 * np.log10(Xr_avg_p)
#print(Xr_avg_db)
noise_avg_db_r = Xr_avg_db - target_snr_db
noise_avg_p_r = 10 ** (noise_avg_db_r / 10)
mean_noise = 0
noise_r = np.random.normal(mean_noise, np.sqrt(noise_avg_p_r), len(Xr))
Xr_ch = Xr + noise_r

Xi_avg_p = np.mean(Xi**2)
Xi_avg_db = 10 * np.log10(Xi_avg_p)
#print(Xi_avg_db)
noise_avg_db_i = Xr_avg_db - target_snr_db
noise_avg_p_i = 10 ** (noise_avg_db_i / 10)
noise_i = np.random.normal(mean_noise, np.sqrt(noise_avg_p_i), len(Xi))
Xi_ch = Xi + noise_i

In [ ]:
plt.plot(Xi_ch, Xr_ch, 'o')
plt.xlabel('I')
plt.ylabel('Q')
plt.title('16-QAM')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for g in np.unique(Symbols_tx):
    ix = np.where(Symbols_tx == g)
    ax.scatter(Xr_ch[ix], Xi_ch[ix], label = g, s = 100)
ax.legend()
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Datos')
plt.show()

# Demodulación

In [ ]:
def demapper_sym(symbols_I, symbols_Q, Ns, threshold = 2.0):
   symbol = []
   for i in range(Ns):
        if symbols_I[i] <= -threshold and symbols_Q[i] >= threshold: #-3+3j
            symbol.append(0)
        elif symbols_I[i] <= -threshold and symbols_Q[i] >= 0 and symbols_Q[i] <= threshold: #-3+1j
            symbol.append(1)
        elif symbols_I[i] <= -threshold and symbols_Q[i] <= 0 and symbols_Q[i] >= -threshold: #-3-j
            symbol.append(3)
        elif symbols_I[i] <= -threshold and symbols_Q[i] <= -threshold: #-3-3j
            symbol.append(2)
        elif symbols_I[i] >= -threshold and symbols_I[i] <= 0 and symbols_Q[i] >= threshold: #-1+3j
            symbol.append(4)
        elif symbols_I[i] >= -threshold and symbols_I[i] <= 0 and symbols_Q[i] >= 0 and symbols_Q[i] <= threshold: #-1+j
            symbol.append(5)
        elif symbols_I[i] >= -threshold and symbols_I[i] <= 0 and symbols_Q[i] <= 0 and symbols_Q[i] >= -threshold: #-1-j
            symbol.append(7)
        elif symbols_I[i] >= -threshold and symbols_I[i] <= 0 and symbols_Q[i] <= -threshold: #-1-3j
            symbol.append(6)
        elif symbols_I[i] >= 0 and symbols_I[i] <= threshold and symbols_Q[i] >= threshold: #1+3j
            symbol.append(12)
        elif symbols_I[i] >= 0 and symbols_I[i] <= threshold and symbols_Q[i] >= 0 and symbols_Q[i] <= threshold: #1+j
            symbol.append(13)
        elif symbols_I[i] >= 0 and symbols_I[i] <= threshold and symbols_Q[i] <= 0 and symbols_Q[i] >= -threshold: #1-j
            symbol.append(15)
        elif symbols_I[i] >= 0 and symbols_I[i] <= threshold and symbols_Q[i] <= -threshold: #1-3j
            symbol.append(14)
        elif symbols_I[i] >= threshold and symbols_Q[i] >= threshold: #3+3j
            symbol.append(8)
        elif symbols_I[i] >= threshold and symbols_Q[i] >= 0 and symbols_Q[i] <= threshold: #3+1j
            symbol.append(9)
        elif symbols_I[i] >= threshold and symbols_Q[i] <= 0 and symbols_Q[i] >= -threshold: #3-1j
            symbol.append(11)
        elif symbols_I[i] >= threshold and symbols_Q[i] <= -threshold: #3-3j
            symbol.append(10)
   return(symbol)

In [ ]:
sym_rx= demapper_sym(Xr_ch, Xi_ch, len(Xr_ch), threshold = 2.0)
print(sym_rx[0:10])


In [ ]:
def SER_f(sym_rx, Symbols_tx, Ns):
    error = 0
    for i in range(Ns):
        if sym_rx[i] == Symbols_tx[i]:
            error = error
        else:
            error= error + 1
    SER = error/Ns
    return([SER, error])

In [ ]:
SER, error = SER_f(sym_rx, Symbols_tx, Ns)
print(SER)
print(error)

In [ ]:
#Comparación Knn vs Tradicional
from numpy import log10
list=[1,3,5,7,9]
for n in list:
  x=[]
  y=[]
  y2=[]
  for i in range(20):
    target_snr_db = i+1
    Xr_avg_p = np.mean(Xr**2)
    Xr_avg_db = 10 * np.log10(Xr_avg_p)
    noise_avg_db_r = Xr_avg_db - target_snr_db
    noise_avg_p_r = 10 ** (noise_avg_db_r / 10)
    mean_noise = 0
    noise_r = np.random.normal(mean_noise, np.sqrt(noise_avg_p_r), len(Xr))
    Xr_ch = Xr + noise_r
    Xi_avg_p = np.mean(Xi**2)
    Xi_avg_db = 10 * np.log10(Xi_avg_p)
    noise_avg_db_i = Xr_avg_db - target_snr_db
    noise_avg_p_i = 10 ** (noise_avg_db_i / 10)
    noise_i = np.random.normal(mean_noise, np.sqrt(noise_avg_p_i), len(Xi))
    Xi_ch = Xi + noise_i
    def SER(sym_rx, Symbols_tx, Ns):
      error = 0
      for i in range(Ns):
          if sym_rx[i] == Symbols_tx[i]:
              error = error
          else:
              error= error + 1
      SER = error/Ns
      return([SER, error])
    sym_rx= demapper_sym(Xr_ch, Xi_ch, len(Xr_ch), threshold = 2.0)
    SER, error = SER(sym_rx, Symbols_tx, Ns)
    y=np.append(y,log10(SER)) 

    X_ = np.transpose(np.array([Xr_ch,Xi_ch]))
    y_= Symbols_tx
    X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3) # 70% training and 30% test
    k= n+1 # número de vecinos
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    model.score(X_train, y_train)
    y_pred = model.predict(X_test)
    SER_test, error_test = SER_f(y_pred, y_test, len(y_pred))
    x.append(i+1) 
    y2=np.append(y2,log10(SER_test))

  plt.plot(x, y, 'o',label = "Demodulación")
  plt.plot(x, y2, 'o',label = f"KNN {n+1}")
  plt.xlabel('SNR (dB)')
  plt.legend()
  plt.ylabel('SER')
  plt.show()

In [ ]:
# BER 
y_pred_bin=[]
y_test_bin=[]
count=0
for i in range(10000):
  y_pred_bin.append(bin(sym_rx[i]))
  y_test_bin.append(bin(Symbols_tx[i]))
  if bin(sym_rx[i])!=bin(Symbols_tx[i]):
    count=count+1
    # print("Error :"+ bin(Symbols_tx[i]) + ", transmitido: "+bin(sym_rx[i]))

# Demodulación usando KNN

In [ ]:
X = np.transpose(np.array([Xr_ch,Xi_ch]))
y= Symbols_tx
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test
k= 10 # número de vecinos
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train, y_train)
model.score(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(y_pred[0:10])
print(y_test[0:10])
SER_test, error_test = SER_f(y_pred, y_test, len(y_pred))
print(SER_test)
print(error_test)

In [ ]:
#BER
y_pred_bin=[]
y_test_bin=[]
count=0
for i in range(1000):
  y_pred_bin.append(bin(y_pred[i]))
  y_test_bin.append(bin(y_test[i]))
  if bin(y_pred[i])!=bin(y_test[i]):
    count=count+1
    # print("Error :"+ bin(y_pred[i]) + ", transmitido: "+bin(y_test[i]))
#print(count)

# Demodulación usando SVM


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training & 30% test
model = SVC()
model.fit(X_train, y_train)
model.score(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01, 0.001, 0.0001]}
grid = GridSearchCV(SVC(), param_grid, verbose = 3)
grid.fit(X_train, y_train)
grid.best_params_
grid.best_estimator_
grid_pred = grid.predict(X_test)

In [ ]:
#Comparación SVM vs Knn vs Tradicional
from numpy import log10
list=[1,3,5,7,9]
for n in list:
  x=[]
  y=[]
  y2=[]
  y3=[]
  for i in range(20):
    target_snr_db = i+1
    Xr_avg_p = np.mean(Xr**2)
    Xr_avg_db = 10 * np.log10(Xr_avg_p)
    noise_avg_db_r = Xr_avg_db - target_snr_db
    noise_avg_p_r = 10 ** (noise_avg_db_r / 10)
    mean_noise = 0
    noise_r = np.random.normal(mean_noise, np.sqrt(noise_avg_p_r), len(Xr))
    Xr_ch = Xr + noise_r
    Xi_avg_p = np.mean(Xi**2)
    Xi_avg_db = 10 * np.log10(Xi_avg_p)
    noise_avg_db_i = Xr_avg_db - target_snr_db
    noise_avg_p_i = 10 ** (noise_avg_db_i / 10)
    noise_i = np.random.normal(mean_noise, np.sqrt(noise_avg_p_i), len(Xi))
    Xi_ch = Xi + noise_i
    def SER(sym_rx, Symbols_tx, Ns):
      error = 0
      for i in range(Ns):
          if sym_rx[i] == Symbols_tx[i]:
              error = error
          else:
              error= error + 1
      SER = error/Ns
      return([SER, error])
    sym_rx= demapper_sym(Xr_ch, Xi_ch, len(Xr_ch), threshold = 2.0)
    SER, error = SER(sym_rx, Symbols_tx, Ns)
    y=np.append(y,log10(SER)) 

    X_ = np.transpose(np.array([Xr_ch,Xi_ch]))
    y_= Symbols_tx
    X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3) # 70% training and 30% test
    k= n+1 # número de vecinos
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    model.score(X_train, y_train)
    y_pred = model.predict(X_test)
    SER_test, error_test = SER_f(y_pred, y_test, len(y_pred))
    x.append(i+1) 
    y2=np.append(y2,log10(SER_test))

    X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_, y_, test_size=0.3) # 70% training & 30% test
    model = SVC()
    model.fit(X_train_svm, y_train_svm)
    model.score(X_train_svm, y_train_svm)
    y_pred_svm = model.predict(X_test_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_svm, y_pred_svm))
    param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose = 3)
    grid.fit(X_train_svm, y_train_svm)
    grid.best_params_
    grid.best_estimator_
    grid_pred = grid.predict(X_test_svm)
    SER_test_svm, error_test_svm = SER_f(grid_pred, y_test_svm, len(y_pred))
    y3=np.append(y3,log10(SER_test_svm))


  plt.plot(x, y, 'o',label = "Demodulación")
  plt.plot(x, y2, '+',label = f"KNN {n+1}")
  plt.plot(x, y3, '*',label = f"SVM {best_params_}")
  plt.xlabel('SNR (dB)')
  plt.legend()
  plt.ylabel('SER')
  plt.show()

[CV 2/5] END ......................C=1, gamma=1;, score=0.363 total time=   2.8s
[CV 3/5] END ......................C=1, gamma=1;, score=0.369 total time=   2.8s
[CV 4/5] END ......................C=1, gamma=1;, score=0.349 total time=   2.7s
[CV 5/5] END ......................C=1, gamma=1;, score=0.349 total time=   2.8s
[CV 1/5] END ....................C=1, gamma=0.1;, score=0.366 total time=   1.6s
[CV 2/5] END ....................C=1, gamma=0.1;, score=0.372 total time=   1.6s
[CV 3/5] END ....................C=1, gamma=0.1;, score=0.379 total time=   2.5s
[CV 4/5] END ....................C=1, gamma=0.1;, score=0.356 total time=   1.8s
[CV 5/5] END ....................C=1, gamma=0.1;, score=0.360 total time=   1.6s
[CV 1/5] END ...................C=1, gamma=0.01;, score=0.362 total time=   1.6s
[CV 2/5] END ...................C=1, gamma=0.01;, score=0.369 total time=   1.6s
[CV 3/5] END ...................C=1, gamma=0.01;, score=0.380 total time=   1.6s
[CV 4/5] END ...............